<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Natural Language Processing</h1>
<h1>Sequence Modeling</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import gzip
from collections import Counter
from pprint import pprint
import time

import random

import pandas as pd
import numpy as np
np.random.seed(42)

import matplotlib
import matplotlib.pyplot as plt 

from tqdm.notebook import tqdm
tqdm.pandas()

RANDOM_SEED = 123
np.random.seed(RANDOM_SEED)

import torch
torch.manual_seed(RANDOM_SEED)

import torch.nn as nn
import torch.nn.functional as F

import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import torchview
from torchview import draw_graph

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 9c8c00758f3a2fa8e55e08f5aad405a157ca5dd2

pandas    : 2.2.3
matplotlib: 3.8.0
torchtext : 0.6.0
torch     : 2.3.0
numpy     : 1.26.4
watermark : 2.4.3
torchview : 0.2.6



Load default figure style

In [3]:
plt.style.use('./d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Load the Dataset

Define the text and label fields

In [4]:
TEXT = Field(tokenize='spacy', tokenizer_language='en_core_web_sm')
LABEL = torchtext.data.LabelField(dtype=torch.long)

Load the dataset with the proper schema

In [5]:
fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = TabularDataset(path='data/IMDB Dataset.csv', format='csv', skip_header=True, fields=fields)

Split the dataset into training, validation, and test partitions:

In [6]:
X_train, X_test = dataset.split(split_ratio=[0.8, 0.2],random_state=random.seed(RANDOM_SEED))
X_train, X_validation = X_train.split(split_ratio=[0.85, 0.15],random_state=random.seed(RANDOM_SEED))

In [7]:
print('Shape of training data:', len(X_train))
print('Shape of test data:', len(X_test))
print('Shape of validation data:', len(X_validation))

Shape of training data: 34000
Shape of test data: 10000
Shape of validation data: 6000


## Build Vocabulary

Build the vocabulary based on the top "vocabulary_size" words:

In [8]:
vocabulary_size = 20_000

In [9]:
TEXT.build_vocab(X_train, max_size=vocabulary_size)
LABEL.build_vocab(X_train)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


## Define Data Loaders

In [10]:
batch_size = 128
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [11]:
train_loader, valid_loader, test_loader = BucketIterator.splits((X_train, X_validation, X_test),
         batch_size=batch_size,
         sort_within_batch=False,
         sort_key=lambda x: len(x.TEXT_COLUMN_NAME),
         device=device
    )

Finally, we are ready to define our models

# Simple RNN model

As the structure of the model is the same for RNN, GRU, and LSTMs, we implement just one of them for demonstration purposes. We can easily plugin another type of layer as needed

In [12]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_size, hidden_size, output_size):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size)        
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, text):        
        embedded = self.embedding(text)
        output, (hidden, cell) = self.rnn(embedded)
        hidden.squeeze_(0)        
        output = self.fc(hidden)
        
        return output

In [13]:
embedding_size = 128
hidden_size = 256
num_classes = 2

In [14]:
model = RNN(len(TEXT.vocab), # vocabulary_size + 2 for <pad> and <unk>
            embedding_size,
            hidden_size,
            num_classes 
           )

In [15]:
print(model)

RNN(
  (embedding): Embedding(20002, 128)
  (rnn): LSTM(128, 256)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)


In [16]:
learning_rate = 0.005
n_epochs = 15

In [17]:
model = model.to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)

## Training

In [18]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            loss = F.cross_entropy(logits, targets)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
            
    return correct_pred.float()/num_examples * 100, loss

Based on this [notebook](https://github.com/rasbt/stat453-deep-learning-ss21/tree/main/L15)

In [ ]:
start_time = time.time()

history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'epochs': n_epochs
    }

for epoch in range(n_epochs):
    model.train()
    for batch_idx, batch_data in tqdm(enumerate(train_loader), 
                                      total=len(train_loader), 
                                      desc='Epoch: %u' % (epoch+1),
                                      leave=True):
        
        text = batch_data.TEXT_COLUMN_NAME.to(device)
        labels = batch_data.LABEL_COLUMN_NAME.to(device)

        # Forward pass
        logits = model(text)
        
        # Back propagation
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        
        # update weights
        optimizer.step()
        
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{n_epochs:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, device)[0]:.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, device)[0]:.2f}%')
    
    history['train_loss'].append(loss.to('cpu').detach().numpy())
    history['train_acc'].append(compute_accuracy(model, train_loader, device)[0].to('cpu').detach().numpy())
    
    val_acc, val_loss = compute_accuracy(model, valid_loader, device)
    
    history['val_acc'].append(val_acc.to('cpu').detach().numpy())
    history['val_loss'].append(val_loss.to('cpu').detach().numpy())

    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, device)[0]:.2f}%')

Epoch: 1:   0%|          | 0/266 [00:00<?, ?it/s]

Epoch: 001/015 | Batch 000/266 | Loss: 0.7136
Epoch: 001/015 | Batch 050/266 | Loss: 0.6994
Epoch: 001/015 | Batch 100/266 | Loss: 0.6898
Epoch: 001/015 | Batch 150/266 | Loss: 0.6926
Epoch: 001/015 | Batch 200/266 | Loss: 0.6961
Epoch: 001/015 | Batch 250/266 | Loss: 0.6951
training accuracy: 49.94%
valid accuracy: 49.82%
Time elapsed: 1.06 min


Epoch: 2:   0%|          | 0/266 [00:00<?, ?it/s]

Epoch: 002/015 | Batch 000/266 | Loss: 0.6954
Epoch: 002/015 | Batch 050/266 | Loss: 0.6933
Epoch: 002/015 | Batch 100/266 | Loss: 0.6932
Epoch: 002/015 | Batch 150/266 | Loss: 0.6926
Epoch: 002/015 | Batch 200/266 | Loss: 0.6930
Epoch: 002/015 | Batch 250/266 | Loss: 0.6915
training accuracy: 50.14%
valid accuracy: 49.38%
Time elapsed: 2.04 min


Epoch: 3:   0%|          | 0/266 [00:00<?, ?it/s]

...

Epoch: 013/015 | Batch 000/266 | Loss: 0.1361
Epoch: 013/015 | Batch 050/266 | Loss: 0.0600
Epoch: 013/015 | Batch 100/266 | Loss: 0.0953
Epoch: 013/015 | Batch 150/266 | Loss: 0.1170
Epoch: 013/015 | Batch 200/266 | Loss: 0.2049
Epoch: 013/015 | Batch 250/266 | Loss: 0.0634
training accuracy: 97.55%
valid accuracy: 86.08%
Time elapsed: 11.47 min


Epoch: 14:   0%|          | 0/266 [00:00<?, ?it/s]

Epoch: 014/015 | Batch 000/266 | Loss: 0.0310
Epoch: 014/015 | Batch 050/266 | Loss: 0.1754
Epoch: 014/015 | Batch 100/266 | Loss: 0.0735
Epoch: 014/015 | Batch 150/266 | Loss: 0.1243
Epoch: 014/015 | Batch 200/266 | Loss: 0.1415
Epoch: 014/015 | Batch 250/266 | Loss: 0.0495
training accuracy: 98.08%
valid accuracy: 86.80%
Time elapsed: 12.25 min


Epoch: 15:   0%|          | 0/266 [00:00<?, ?it/s]

Epoch: 015/015 | Batch 000/266 | Loss: 0.0661


In [ ]:
def plot_history(history):    
    fig, ax_lst = plt.subplots(1, 2, sharex=True, sharey=False)
    
    epochs = range(history['epochs'])
    
    ax_lst[0].plot(epochs, history['train_loss'], label='Training')
    ax_lst[0].plot(epochs, history['val_loss'], label='Testing')
    ax_lst[0].set_ylabel('Loss')
    ax_lst[0].set_xlabel('Epoch')
    ax_lst[0].set_xticks(epochs)
    
    ax_lst[1].plot(epochs, history['train_acc'], label='Training')
    ax_lst[1].plot(epochs, history['val_acc'], label='Testing')
    ax_lst[1].set_ylabel('Accuracy')
    ax_lst[1].set_xlabel('Epoch')
    ax_lst[1].set_xticks(epochs)

    ax_lst[1].legend()
    
    fig.tight_layout()

In [ ]:
plot_history(history)

<div style="width: 100%; overflow: hidden;">
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</div>